In [154]:
"""
Created on 14th Jan,2025

@author: Ayush Goel
"""

import numpy as np
import pandas as pd
import yfinance as yf
from datetime import date
from datetime import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import statsmodels.api as sm

In [155]:
df = pd.read_excel(r"H:\Learning Material\Interest Rate Models\Nelson Siegel _Svensson\Nelson-Siegel_Svensson.xlsx",sheet_name="OIS Curve_NSS",skiprows=13)



In [156]:
df_22_09_2011 = df.iloc[:,:4]
df_22_09_2011.drop(columns=["Term"])
df_22_09_2011.set_index("Term(Years)",inplace=True)
df_22_09_2011





,Term,Rate%,NSS
Term(Years),,,
0.082192,30,0.9231,1.077513
0.164384,60,0.7720,0.642154
0.246575,90,0.7105,0.513449
0.493151,180,0.5995,0.432880
0.739726,270,0.5710,0.452648
1.000000,365,0.5685,0.498808
1.246575,455,0.5743,0.550880
1.493151,545,0.5879,0.606078
1.739726,635,0.6127,0.662345


### NSS Model (What we are estimating):-

The NSS yield curve is

 $y(\tau) = \beta_{0} + \beta_1(\frac{1-exp^(-\frac{\tau}{\lambda_1})}{(\frac{\tau}{\lambda_1})}) + \beta_2(\frac{1-exp^(-\frac{\tau}{\lambda_1})}{(\frac{\tau}{\lambda_1})} - exp^{\frac{\tau}{\lambda_1}})  + \beta_3(\frac{1-exp^(-\frac{\tau}{\lambda_2})}{(\frac{\tau}{\lambda_2})} - exp^{\frac{\tau}{\lambda_2}})$ 
 
 where:-
 
 - $\tau$ = ***maturity***
 - $y(\tau)$ = ***observed yield***
 - $\beta_0,\beta_1,\beta_2,\beta_3$ = ***factors***
 - $\lambda_1,\lambda_2$ = ***decay parameters***

#### Why OLS Works Here

If #\lambda_1,\lambda_2 are fixed, the model is:

$𝑦 = 𝑋(\lambda_1,\lambda_2)\beta + \epsilon$
This is linear in β, so we can estimate β via ordinary least squares by minimizing:

$\sum_{i=1}^{n}(y-\vec{y}_{i})^{2}$




⚠️ **Important:**

NSS is nonlinear overall

- ***But linear conditional on λ***

***Standard practice:***

- Fix λ

- Estimate β via OLS

- Optionally optimize λ separately


In [157]:
def nss_loadings(tau,lambda1,lambda2):
    f_0 = np.ones_like(tau)
    f_1 = (1-np.exp(-tau/lambda1))/(tau/lambda1)
    f_2 = (1-np.exp(-tau/lambda1))/(tau/lambda1) - np.exp(-tau/lambda1)
    f_3 = (1-np.exp(-tau/lambda2))/(tau/lambda2) - np.exp(-tau/lambda2)
    X = np.column_stack([f_0,f_1,f_2,f_3])
    return X

def nss_ols_method_1(tau,y,lambda1,lambda2):
    X = nss_loadings(tau,lambda1,lambda2)
    model = sm.OLS(y,X).fit()
    OLS = model.params
    #OLS = np.linalg.inv(X.T@X)@X.T@y
    beta = np.array([OLS[0],OLS[1],OLS[2],OLS[3]])
    return beta

def nss_ols_method_2(tau,y,lambda1,lambda2):
    X = nss_loadings(tau,lambda1,lambda2)
    OLS = np.linalg.lstsq(X,y,rcond=None)[0]
    beta = np.array([OLS[0],OLS[1],OLS[2],OLS[3]])
    return beta
    
def nss_yield(tau,y,lambda1,lambda2):
    X = nss_loadings(tau,lambda1,lambda2)
    beta = nss_ols_method_1(tau,y,lambda1,lambda2)
    y_hat = X@beta 
    return y_hat

def nss_sse(tau,y,lambda1,lambda2):
    y_hat = nss_yield(tau,y,lambda1,lambda2)
    residual = np.sum((y-y_hat)**2)
    return residual

In [158]:
X = nss_loadings(df_22_09_2011.index.to_numpy(),1.289e+00,1.289e+00)
print(X)
beta = nss_ols_method_1(df_22_09_2011.index.to_numpy(),df_22_09_2011["Rate%"].to_numpy(),1.289e+00,1.289e+00)
print(beta.shape)
bea_1 = nss_ols_method_2(df_22_09_2011.index.to_numpy(),df_22_09_2011["Rate%"].to_numpy(),1.289e+00,1.289e+00)
print(bea_1)

y_hat = nss_yield(df_22_09_2011.index.to_numpy(),df_22_09_2011["Rate%"].to_numpy(),1.289e+00,1.289e+00)
print(y_hat)
residual = nss_sse(df_22_09_2011.index.to_numpy(),df_22_09_2011["Rate%"].to_numpy(),1.289e+00,1.289e+00)
residual

[[1.         0.96878498 0.03055858 0.03055858]
 [1.         0.93886232 0.05859353 0.05859353]
 [1.         0.91017194 0.08428052 0.08428052]
 [1.         0.83093757 0.14884093 0.14884093]
 [1.         0.76090012 0.19756235 0.19756235]
 [1.         0.69562486 0.23528728 0.23528728]
 [1.         0.64090515 0.2607163  0.2607163 ]
 [1.         0.59221137 0.27821665 0.27821665]
 [1.         0.54878145 0.28945591 0.28945591]
 [1.         0.50792356 0.29601287 0.29601287]
 [1.         0.47334508 0.29832986 0.29832986]
 [1.         0.44228508 0.29774151 0.29774151]
 [1.         0.41431977 0.29494248 0.29494248]
 [1.         0.38775249 0.29020204 0.29020204]
 [1.         0.3650463  0.28448022 0.28448022]
 [1.         0.34445449 0.27791565 0.27791565]
 [1.         0.32573629 0.27078244 0.27078244]
 [1.         0.307779   0.26287286 0.26287286]
 [1.         0.29228119 0.25519359 0.25519359]
 [1.         0.27809384 0.24746351 0.24746351]
 [1.         0.26507687 0.23977955 0.23977955]
 [1.         

0.15263268790896564

### "Why This is OLS NSS"

- Minimizes the ***sum of squared error***
- Linear Regression conditional on $\lambda$
- stable and interpretable

### Important Limitation

***OLS does not estimate $\lambda$ automatically.***

To estimate $\lambda$ as well, you must use nonlinear optimization:

$$\min_{\beta,\lambda_1,\lambda_2}\Sigma(y-yNSS)^2$$

(using scipy.optimize.minimize)

#### Key Takeaway

Nelson–Siegel–Svensson is linear in β and nonlinear in λ.OLS is valid once λ is fixed.

In [168]:
from scipy.optimize import minimize

In [180]:
### Implementing Paramas:-

init_param = [0.001,0.001,0.001,0.001,1,1]
param_bound = [(None,None),(None,None),(None,None),(None,None), (None, None),(None, None)]

def nss_yield_(tau,b0,b1,b2,b3,lambda1,lambda2):
    tau = df_22_09_2011.index.to_numpy()
    f0 = (1-np.exp(-tau/lambda1))/(tau/lambda1)
    f1 = (1-np.exp(-tau/lambda1))/(tau/lambda1) - np.exp(-tau/lambda1)
    f2 = (1-np.exp(-tau/lambda2))/(tau/lambda2) - np.exp(-tau/lambda2)
    #y_hat = b0 + f0*b1 + f1*b2 + f2*b3
    
    return b0 + f0*b1 + f1*b2 + f2*b3

def nss_residual(param,y,tau):
    #b0,b1,b2,b3,lambda1,lambda2 = param
    y = df_22_09_2011["Rate%"].to_numpy()
    #y_hat_ = nss_yield_(tau,*param)
    y_residual = np.sum((y-nss_yield_(tau,*param))**2)
    return y_residual

In [181]:
model = minimize(nss_residual,init_param,args=(y,tau),bounds=param_bound, method="L-BFGS-B")
model

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.15263260558932823
        x: [ 2.702e+00 -1.769e+00 -1.658e+00 -2.328e+00  1.289e+00
             1.289e+00]
      nit: 27
      jac: [ 5.199e-06  2.523e-06  1.044e-06  1.019e-06 -2.934e-06
            -2.198e-06]
     nfev: 210
     njev: 30
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [171]:
y = df_22_09_2011["Rate%"].to_numpy() 
tau = df_22_09_2011.index.to_numpy()
model_revisit = minimize(lambda b: ((y-(b[0] + b[1]*(1-np.exp(-tau/b[4]))/(tau/b[4])+b[2]*(((1-np.exp(-tau/b[4]))/(tau/b[4]))-np.exp(-tau/b[4])) + b[3]*(((1-np.exp(-tau/b[5]))/(tau/b[5]))-np.exp(-tau/b[5]))))**2).sum(),init_param,bounds = param_bound)

In [172]:
model_revisit

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.1526326055893713
        x: [ 2.702e+00 -1.769e+00 -1.658e+00 -2.328e+00  1.289e+00
             1.289e+00]
      nit: 27
      jac: [ 5.260e-06  2.492e-06  1.041e-06  1.049e-06 -3.014e-06
            -2.215e-06]
     nfev: 210
     njev: 30
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [173]:
def nss_yield(tau, b0, b1, b2, b3, l1, l2):
    f1 = (1 - np.exp(-tau / l1)) / (tau / l1)
    f2 = f1 - np.exp(-tau / l1)
    f3 = (1 - np.exp(-tau / l2)) / (tau / l2) - np.exp(-tau / l2)
    return b0 + b1*f1 + b2*f2 + b3*f3


def nss_sse(b, tau, y):
    return np.sum((y - nss_yield(tau, *b))**2)

In [174]:
model = minimize(
    nss_sse,
    init_param,
    args=(tau, y),
    bounds=param_bound,
    method="L-BFGS-B")


In [175]:
model

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.15263260558932823
        x: [ 2.702e+00 -1.769e+00 -1.658e+00 -2.328e+00  1.289e+00
             1.289e+00]
      nit: 27
      jac: [ 5.199e-06  2.523e-06  1.044e-06  1.019e-06 -2.934e-06
            -2.198e-06]
     nfev: 210
     njev: 30
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [182]:
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve

In [ ]:
y = NelsonSiegelSvenssonCurve()

In [185]:
from nelson_siegel_svensson.calibrate import NelsonSiegelSvenssonCurve
import nelson_siegel_svensson as nss

In [187]:
y = nss.calibrate.minimize(nss_residual,init_param,args=(y,tau),bounds=param_bound)

In [188]:
y

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.15263260558932823
        x: [ 2.702e+00 -1.769e+00 -1.658e+00 -2.328e+00  1.289e+00
             1.289e+00]
      nit: 27
      jac: [ 5.199e-06  2.523e-06  1.044e-06  1.019e-06 -2.934e-06
            -2.198e-06]
     nfev: 210
     njev: 30
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>